# Preprocessing + Topic Modeling (Mixed Language Approach)
This notebook serves as a demo for topic modeling GCash App Store Reviews using LDA. The approach of this notebook is to ignore the multi-lingual nature of reviews and move directly to LDA after data cleaning. This notebook will be divided into four parts:

1. Data Cleaning
2. Text Cleaning
3. Modeling (LDA)
4. Exploration of Results

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!ls "/content/drive/My Drive/Data/Reviews/Play Store"

gcash_playstore_reviews_final.csv  paymaya_reviews_playstore.csv
gcash_reviews_playstore.csv


In [ ]:
!pip install emot contractions pyLDAvis lda
!python3 -m spacy download en

In [3]:
# importing libraries
import pandas as pd
import numpy as np 
import regex as re
import datetime
import json

from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import contractions


from spacy.lang.tl.stop_words import STOP_WORDS as tl_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

#nltk
import nltk
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import models
import nltk
import tqdm



# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [4]:
# importing the dataset
df = pd.read_csv('/content/drive/My Drive/Data/Reviews/Play Store/gcash_playstore_reviews_final.csv')

In [5]:
df.shape

(199298, 13)

In [6]:
df.dtypes

_id                     object
app_id                  object
app_name                object
at                      object
content                 object
repliedAt               object
replyContent            object
reviewCreatedVersion    object
reviewId                object
score                    int64
thumbsUpCount            int64
userImage               object
userName                object
dtype: object

In [7]:
df.head(2)

,_id,app_id,app_name,at,content,repliedAt,replyContent,reviewCreatedVersion,reviewId,score,thumbsUpCount,userImage,userName
0,60b76108c5eccca0e375ce55,com.globe.gcash.android,GCash,2021-06-02T18:32:37.000Z,Good for mobile online deals.,NaN,NaN,5.40.0,gp:AOqpTOHa4x04QC93zb0pxQZ-1Q1GwFeysD-XpcDTW78...,1,0,https://play-lh.googleusercontent.com/a/AATXAJ...,Emmanuel Jim Roldan
1,60b76108c5eccca0e375ce56,com.globe.gcash.android,GCash,2021-06-02T18:32:14.000Z,Ilang beses nang naulit na nag load ako nag ba...,NaN,NaN,5.41.0,gp:AOqpTOFOvZwSS0iVvmGbcBKkwFyprROh5KqfcJuI5jg...,1,0,https://play-lh.googleusercontent.com/a-/AOh14...,Harem Tuazon


In [8]:
df['score'].value_counts()

5    118576
1     42537
4     16187
3     12885
2      9113
Name: score, dtype: int64

## Part 1: Data Exploration + Cleaning

In [9]:
# Checking for null values
df.isnull().sum() 

_id                          0
app_id                       0
app_name                     0
at                           0
content                     14
repliedAt               178901
replyContent            178901
reviewCreatedVersion     48259
reviewId                     0
score                        0
thumbsUpCount                0
userImage                    0
userName                     0
dtype: int64

In [10]:
# Dropping unwanted columns [userName, app_name, app_id, developerResponse]
df = df.drop(['_id', 'app_id', 'app_name', 'replyContent', 'reviewCreatedVersion', 'reviewId', 'thumbsUpCount', 'userImage', 'userName', 'repliedAt'], axis=1)

In [11]:
# Splitting datetime column + Checking date range

# Code block below separated the timestamp [[date]] column into Date and Time column respectively
df['Dates'] = pd.to_datetime(df['at']).dt.date
df['Time'] = pd.to_datetime(df['at']).dt.time
df = df.drop('at', axis=1)

In [12]:
df.sort_values(by='Dates', ascending=False).head(1)

,content,score,Dates,Time
0,Good for mobile online deals.,1,2021-06-02,18:32:37


In [13]:
df.sort_values(by='Dates', ascending=True).head(1)

,content,score,Dates,Time
198696,"""Unknown error occurred"" always popping up! Ne...",1,2012-03-26,18:49:57


In the section above, we can see that the dataset ranges from 2012-05-23 to 2021-06-11.

In [14]:
df.dtypes

content    object
score       int64
Dates      object
Time       object
dtype: object

In [15]:
df['word_count'] = df['content'].str.split().str.len()

In [16]:
df.head(5)

,content,score,Dates,Time,word_count
0,Good for mobile online deals.,1,2021-06-02,18:32:37,5.0
1,Ilang beses nang naulit na nag load ako nag ba...,1,2021-06-02,18:32:14,19.0
2,Its a great experience and convenient,4,2021-06-02,18:32:07,6.0
3,Ok na ok sya para sa mga easy transaction lalo...,5,2021-06-02,18:31:31,16.0
4,very helpful and contented,5,2021-06-02,18:31:25,4.0


In [17]:
df2 = df.copy()

In [18]:
df2 = df2.dropna(subset=['content'])

In [19]:
# Filter pandas df for reviews from january 2020 to may 2021
# import datetime
df2 = df2[(df2['Dates']>datetime.date(2020,1,1)) & (df2['Dates']<datetime.date(2021,5,31))] 

In [20]:
df2.shape

(130233, 5)

In [21]:
df2.sort_values(by='word_count')

,content,score,Dates,Time,word_count
102908,😊,5,2020-07-04,17:15:27,1.0
93166,Ok,5,2020-07-27,11:19:36,1.0
93164,good,5,2020-07-27,11:20:27,1.0
93163,good,5,2020-07-27,11:20:38,1.0
29655,good,5,2021-03-15,18:51:20,1.0
...,...,...,...,...,...
26452,You so much and I hope you have a great day an...,5,2021-04-02,16:02:29,114.0
38037,They stole my money. They said they're going t...,1,2021-02-17,15:58:22,117.0
102007,Just got home at the end of the day I love you...,5,2020-07-11,09:43:09,119.0
134248,Mop is a great day and I will be there to help...,5,2020-01-09,06:14:33,122.0


In [22]:
df2['word_count'].value_counts().sort_index().head(10)

1.0     35495
2.0     21638
3.0     10440
4.0      7608
5.0      5841
6.0      4527
7.0      3706
8.0      2985
9.0      2622
10.0     2374
Name: word_count, dtype: int64

In [23]:
df2.loc[df2['word_count'].isin([1])]

,content,score,Dates,Time,word_count
4549,Excellent!,5,2021-05-30,23:45:41,1.0
4555,Good,4,2021-05-30,22:17:59,1.0
4558,Good,5,2021-05-30,22:01:46,1.0
4591,Good,5,2021-05-30,17:33:12,1.0
4593,Good,5,2021-05-30,17:28:39,1.0
...,...,...,...,...,...
134754,Good,5,2020-01-02,12:21:38,1.0
134758,Nice,5,2020-01-02,12:12:53,1.0
134759,Good,5,2020-01-02,12:12:27,1.0
134781,nice,5,2020-01-02,05:42:03,1.0


## Part 2: Text Cleaning

Now that we have done the initial cleaning of the df, let us now proceed to text cleaning for topic modeling purposes.

My approach in this notebook would be as follows:
1. Do the necessary text cleaning steps (remove encodings, lowercase, strip punctutation, expand contractions etc)
2. Remove Filipino stop words and possibly lemmatize Filipino words
3. Remove English stop words and lemmatize
4. Tokenization and other methods

In [24]:
df2.head(4)

,content,score,Dates,Time,word_count
4547,Very good,5,2021-05-30,23:59:34,2.0
4548,So good app to me,5,2021-05-30,23:57:27,5.0
4549,Excellent!,5,2021-05-30,23:45:41,1.0
4550,It is easy to sending money and load by Gcash.,5,2021-05-30,23:45:05,10.0


In [25]:
# Rename content column to final_review + reset index
df2.rename(columns={'content':'final_review'}, inplace=True)

In [26]:
df2 = df2.reset_index(drop=True)

#### Remove emojis and emoticons

In [27]:
# Remove emojis and emoticons

# Code block 1: remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Example
remove_emoji("Have fun with NLP! 😃😃")

'Have fun with NLP! '

In [28]:
# apply the remove emoji function to the review column
df2['final_review'] = df2['final_review'].apply(lambda x: remove_emoji(x))

In [29]:
df2.loc[df2['word_count'].isin([1])]

,final_review,score,Dates,Time,word_count
2,Excellent!,5,2021-05-30,23:45:41,1.0
8,Good,4,2021-05-30,22:17:59,1.0
11,Good,5,2021-05-30,22:01:46,1.0
44,Good,5,2021-05-30,17:33:12,1.0
46,Good,5,2021-05-30,17:28:39,1.0
...,...,...,...,...,...
130198,Good,5,2020-01-02,12:21:38,1.0
130202,Nice,5,2020-01-02,12:12:53,1.0
130203,Good,5,2020-01-02,12:12:27,1.0
130225,nice,5,2020-01-02,05:42:03,1.0


In [30]:
# Code block 2: remove emoticons

# from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)


In [31]:
#Example
remove_emoticons("I used to play Super Mario games :)")

'I used to play Super Mario games '

In [32]:
# Applying remove emoticons code
df2['final_review'] = df2['final_review'].apply(lambda x: remove_emoticons(x))

In [33]:
df2['word_count'] = df2['final_review'].str.split().str.len()

In [34]:
df2['word_count'].value_counts().sort_index().head(6)

0     2810
1    33170
2    21874
3    10145
4     7572
5     5751
Name: word_count, dtype: int64

In [35]:
df2.loc[df2['word_count'].isin([2])]

,final_review,score,Dates,Time,word_count
0,Very good,5,2021-05-30,23:59:34,2
4,Love it,5,2021-05-30,23:33:04,2
9,Nice app,5,2021-05-30,22:11:04,2
13,Verified now,5,2021-05-30,21:13:50,2
14,Can't online,1,2021-05-30,21:13:42,2
...,...,...,...,...,...
130223,Faster transaction,3,2020-01-02,07:39:58,2
130227,Great app,5,2020-01-02,02:25:38,2
130228,nice app,5,2020-01-02,02:08:09,2
130229,Love it!!!,5,2020-01-02,01:22:27,2


#### Remove \n, \t, \r

In [36]:
df2['final_review'][12]

"How dare you charged us for over the counter cash ins that's insane! You guys are such a ripped off! Just because the app is booming doesn't mean you can ripped us off anytime you want now! Bastards "

In [37]:
# Remove \r \n \t
df2['final_review'] = df2['final_review'].replace(r'\r+|\n+|\t+',' ', regex=True)

In [38]:
df2['final_review'][12]

"How dare you charged us for over the counter cash ins that's insane! You guys are such a ripped off! Just because the app is booming doesn't mean you can ripped us off anytime you want now! Bastards "

#### Convert reviews to lowercase

In [39]:
df2['final_review'] = df2['final_review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df2['final_review'].head()

0                                         very good
1                                 so good app to me
2                                        excellent!
3    it is easy to sending money and load by gcash.
4                                           love it
Name: final_review, dtype: object

#### Removing URLS

In [40]:
# import re

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

df2['final_review'] = df2.final_review.apply(remove_urls)
df2.final_review.head()

0                                         very good
1                                 so good app to me
2                                        excellent!
3    it is easy to sending money and load by gcash.
4                                           love it
Name: final_review, dtype: object

#### Expand Contractions

Contractions are words or combinations of words which are shortened by dropping letters and replacing them by an apostrophe. 

Let’s have a look at some examples:
* we’re = we are
* we’ve = we have
* I’d = I would

Note: This step needs to be done before word tokenizer because NLTK word tokenizer has in-built methods for dealing with contractions. However, NLTK word tokenizer's approach separates contractions without expanding. Expanding is a better method than simply separating.

In [41]:
# import contractions

# Example text 
text = ''' She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.'''
  
# creating an empty list 
expanded_words = []     
for word in text.split(): 
  # using contractions.fix to expand the shotened words 
  expanded_words.append(contractions.fix(word))    
    
expanded_text = ' '.join(expanded_words) 
print('Original text: ' + text) 
print('\n') 
print('Expanded_text: ' + expanded_text)

Original text:  She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.


Expanded_text: she will be airport in 30 mins. We are supposed to catch the arrival, are not we? I would love to welcome her personally. it will be an awesome vacation.


In [42]:
df2['final_review'] = df2['final_review'].apply(lambda x: contractions.fix(x))
df2.final_review.sample(5)

5763      i really love the app it is very convenient bu...
122358                                             good app
26384     it is basically useless when you are not verif...
79474     easiest way to pay and get money. just wow mor...
52656                                                  good
Name: final_review, dtype: object

#### Separating good and bad reviews into separate dataframes

In [43]:
df2 = df2.reset_index()

In [44]:
df_goodreviews = df2.loc[df2['score'].isin([5])].copy()
df_badreviews = df2.loc[df2['score'].isin([1])].copy()

In [45]:
print(f'df_good_reviews: {df_goodreviews.shape}')
print(f'df_bad_reviews: {df_badreviews.shape}')

df_good_reviews: (76657, 6)
df_bad_reviews: (28226, 6)


In [46]:
df2['score'].value_counts()

5    76657
1    28226
4    10625
3     8576
2     6149
Name: score, dtype: int64

## Part 3: Modeling

#### Download nltk English stopwords and spacy model (EN for lemmatization)

In [47]:
# Run in python console
# import nltk
nltk.download('stopwords')

# Run in terminal or command prompt
#!python3 -m spacy download |en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#### Prepare Stop Words (English + Tagalog)

Sources for English stopwords: 
1. nltk library

Sources for Tagalog stopwords:
1. Typical Stop Words: https://github.com/stopwords-iso/stopwords-tl/blob/master/stopwords-tl.json
2. Profanity : https://github.com/jromest/filipino-badwords-list/blob/master/src/filipino-badwords-list.js

In [48]:
# from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [49]:
# import json
with open('/content/drive/My Drive/Data/stopwords/tagalog-sw') as file:
    tl_stopwords = json.load(file)

In [50]:
# Extend stopwords to include tagalog + common gcash related words 
stop_words.extend(tl_stopwords)
stop_words.extend(['gcash','g-cash','po', 'app', 'would','can'])
stop_words.extend(['nyo','naman','yung', 'di', 'wala ','mag', 'nag', 'pag', 'kayo', 'lang', 'ung', 'niyo', 'tapos', 'ba', 'mo', 'please', 'pls'])
stop_words.extend(['nga','yan','yun', 'akong', 'inyo', 'ur', 'star', 'nalang', 'kasi', 'talaga'])
stop_words.extend(['nmn', 'nlng','un','blah', 'tru', 'pwede','like','one','globe', 'guys', 'even', 'seems', 'hi', 'guess'])

#### Tokenize and Clean Up Text some more

In [51]:
# Check columns
df_badreviews.columns

Index(['index', 'final_review', 'score', 'Dates', 'Time', 'word_count'], dtype='object')

In [52]:
# Convert the review columns into a list
bad_reviews = df_badreviews.final_review.values.tolist()
good_reviews = df_goodreviews.final_review.values.tolist()

In [53]:
# Define a function to tokenize and remove punctuations
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

br_words = list(sent_to_words(bad_reviews))
gr_words = list(sent_to_words(good_reviews))

#### Create Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are *min_count* and *threshold*. The higher the values of these param, the harder it is for words to be combined to bigrams.

###### Bigram and Trigram for good reviews

In [54]:
# Build the bigram and trigram models
g_bigram = gensim.models.Phrases(gr_words, min_count=5, threshold=80) # higher threshold fewer phrases.
g_trigram = gensim.models.Phrases(g_bigram[gr_words], threshold=80)  

# Faster way to get a sentence clubbed as a trigram/bigram
g_bigram_mod = gensim.models.phrases.Phraser(g_bigram)
g_trigram_mod = gensim.models.phrases.Phraser(g_trigram)

# See trigram example
print(g_trigram_mod[g_bigram_mod[gr_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['very', 'good']


##### Bigram and Trigam for bad reviews

In [55]:
# Build the bigram and trigram models
b_bigram = gensim.models.Phrases(br_words, min_count=5, threshold=80) # higher threshold fewer phrases.
b_trigram = gensim.models.Phrases(b_bigram[br_words], threshold=80)  

# Faster way to get a sentence clubbed as a trigram/bigram
b_bigram_mod = gensim.models.phrases.Phraser(b_bigram)
b_trigram_mod = gensim.models.phrases.Phraser(b_trigram)

# See trigram example
print(b_trigram_mod[b_bigram_mod[br_words[3]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['super', 'hustle', 'everytime', 'tried', 'to', 'load', 'my', 'phone', 'its', 'always', 'error', 'what', 'happened', 'to', 'you', 'gcash', 'poor', 'service']


#### Remove Stopwords, Make Bigrams and Lemmatize (Eng. words only)

##### Functionalizing the steps

I have excluded lemmatization as it brought out errant results when added to tagalog corpus.

In [56]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def g_make_bigrams(texts):
    return [g_bigram_mod[doc] for doc in texts]

def g_make_trigrams(texts):
    return [g_trigram_mod[g_bigram_mod[doc]] for doc in texts]

def b_make_bigrams(texts):
    return [b_bigram_mod[doc] for doc in texts]

def b_make_trigrams(texts):
    return [b_trigram_mod[b_bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

##### Applying the functions to tokenized bad and good reviews


In [57]:
# APPLYING TO TOKENIZED GOOD REVIEWS

# Remove Stop Words
gr_words_nostops = remove_stopwords(gr_words)

# ### REMOVED LEMMATIZATION COS IT AFFECTS THE TAGALOG WORDS / TAGALOG WORDS ARE RENDERED INEFFECTIVE
# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# gr_lemmatized = lemmatization(gr_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


# Form Bigrams
gr_words_bigrams = g_make_bigrams(gr_words_nostops)

# Form Trigrams
gr_words_trigrams = g_make_trigrams(gr_words_bigrams)


In [58]:
gr_words_trigrams[13]

['nice', 'excellent']

In [59]:
# APPLYING TO TOKENIZED BAD REVIEWS

# Remove Stop Words
br_words_nostops = remove_stopwords(br_words)

# ## REMOVED LEMMATIZATION COS IT AFFECTS THE TAGALOG WORDS / TAGALOG WORDS ARE RENDERED INEFFECTIVE
# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# br_lemmatized = lemmatization(br_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Form Bigrams
br_words_bigrams = b_make_bigrams(br_words_nostops)

br_words_trigrams = b_make_trigrams(br_words_bigrams)




In [60]:
br_words_trigrams[3]


['super',
 'hustle',
 'everytime',
 'tried',
 'load',
 'phone',
 'always',
 'error',
 'happened',
 'poor',
 'service']

#### Create the Dictionary and Corpus needed for Topic Modeling

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

##### Good Reviews -  Dictionary and Corpus

In [61]:
# Create Dictionary
g_id2word = corpora.Dictionary(gr_words_trigrams)
g_id2word.filter_extremes(no_below=10, no_above=0.35)
g_id2word.compactify()
# Create Corpus
g_texts = gr_words_trigrams

# Term Document Frequency (doc2bow) and TFIDF

tfidf   = gensim.models.TfidfModel(dictionary=g_id2word, normalize=True)
g_corpus = [tfidf[g_id2word.doc2bow(text)] for text in g_texts]

# View
#print(g_corpus[:1])

##### Bad Reviews -  Dictionary and Corpus

In [62]:
# Create Dictionary
b_id2word = corpora.Dictionary(br_words_trigrams)
b_id2word.filter_extremes(no_below=10, no_above=0.35)
b_id2word.compactify()

# Create Corpus
b_texts = br_words_trigrams

# Term Document Frequency (doc2bow) and TFIDF
tfidf   = gensim.models.TfidfModel(dictionary=b_id2word, normalize=True)
b_corpus = [tfidf[g_id2word.doc2bow(text)] for text in b_texts]

# View
print(b_corpus[:1])

[[(12, 0.2016938730296311), (61, 0.1521936926708742), (62, 0.2173288297073219), (87, 0.146200757981325), (140, 0.10629712056641898), (142, 0.14360189446406446), (212, 0.2530817371612276), (228, 0.19137952444366169), (243, 0.19137952444366169), (273, 0.2530817371612276), (348, 0.16313661782254063), (460, 0.2552667037161239), (477, 0.18486672954027655), (611, 0.26282128023830725), (654, 0.21190039221661913), (656, 0.22379704892130836), (732, 0.21127917788455064), (875, 0.2265812387511781), (1005, 0.21883375440673403), (1011, 0.22040870029182527), (1064, 0.2285732974838136), (1082, 0.20722328056597022), (1245, 0.2379725841168609)]]


#### Building the Topic Model

##### Optizing Number of Topics for Good Reviews

In [63]:
# Build initial LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=g_corpus,
                                           id2word=g_id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [64]:
print(json.dumps(lda_model.print_topics(), indent=1))

[
 [
  0,
  "0.373*\"nice\" + 0.209*\"great\" + 0.120*\"apps\" + 0.097*\"awesome\" + 0.035*\"thank\" + 0.026*\"thanks\" + 0.017*\"reliable\" + 0.011*\"need\" + 0.008*\"update\" + 0.008*\"stars\""
 ],
 [
  1,
  "0.175*\"convenient\" + 0.122*\"helpful\" + 0.070*\"usefull\" + 0.057*\"fast\" + 0.049*\"bills\" + 0.044*\"wow\" + 0.039*\"super\" + 0.028*\"cash\" + 0.025*\"paying\" + 0.021*\"online\""
 ],
 [
  2,
  "0.473*\"good\" + 0.119*\"useful\" + 0.111*\"ok\" + 0.095*\"love\" + 0.062*\"amazing\" + 0.026*\"satisfied\" + 0.014*\"experience\" + 0.008*\"perfect\" + 0.006*\"tnx\" + 0.005*\"gamitin\""
 ],
 [
  3,
  "0.266*\"excellent\" + 0.057*\"happy\" + 0.044*\"far\" + 0.042*\"convinient\" + 0.041*\"hassle\" + 0.037*\"job\" + 0.037*\"cool\" + 0.026*\"time\" + 0.023*\"excelent\" + 0.023*\"thumbs\""
 ],
 [
  4,
  "0.097*\"best\" + 0.048*\"load\" + 0.032*\"application\" + 0.028*\"help\" + 0.026*\"much\" + 0.023*\"okay\" + 0.022*\"payment\" + 0.020*\"bank\" + 0.020*\"buy\" + 0.017*\"lot\""
 ],
 [

In [65]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=gr_words_trigrams, dictionary=g_id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [66]:
# #Optimizing topic models

# import numpy as np
# import tqdm
# grid = {}
# grid['Validation_Set'] = {}
# # Topics range
# min_topics = 2
# max_topics = 8
# step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
# # Alpha parameter
# alpha = list(np.arange(0.01, 1, 0.3))
# alpha.append('symmetric')
# alpha.append('asymmetric')
# # Beta parameter
# beta = list(np.arange(0.01, 1, 0.3))
# beta.append('symmetric')
# # Validation sets
# num_of_docs = len(g_corpus)
# corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
#                # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#                #gensim.utils.ClippedCorpus(g_corpus, num_of_docs*0.75), 
#                g_corpus]
# corpus_title = ['75% Corpus', '100% Corpus']
# model_results = {'Validation_Set': [],
#                  'Topics': [],
#                  'Alpha': [],
#                  'Beta': [],
#                  'Coherence': []
#                 }
# # Can take a long time to run
# if 1 == 1:
#     pbar = tqdm.tqdm(total=540)
    
#     # iterate through validation corpuses
#     for i in range(len(corpus_sets)):
#         # iterate through number of topics
#         for k in topics_range:
#             # iterate through alpha values
#             for a in alpha:
#                 # iterare through beta values
#                 for b in beta:
#                     # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=g_id2word, 
#                                                   k=k, a=a, b=b)
#                     # Save the model results
#                     model_results['Validation_Set'].append(corpus_title[i])
#                     model_results['Topics'].append(k)
#                     model_results['Alpha'].append(a)
#                     model_results['Beta'].append(b)
#                     model_results['Coherence'].append(cv)
                    
#                     pbar.update(1)
#     pd.DataFrame(model_results).to_csv('gcash_gr_lda_tuning_results.csv', index=False)
#     pbar.close()

In [67]:
# model_cv_df_gr = pd.DataFrame(model_results) 
# model_cv_df_gr.sort_values(by='Coherence',ascending=False)

In [68]:
# Build optimized LDA model

lda_model_experimental = gensim.models.LdaMulticore(corpus=g_corpus,
                                           id2word=g_id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [69]:
print(json.dumps(lda_model_experimental.print_topics(), indent=1))

[
 [
  0,
  "0.424*\"nice\" + 0.237*\"great\" + 0.136*\"apps\" + 0.064*\"best\" + 0.033*\"satisfied\" + 0.018*\"job\" + 0.007*\"ever\" + 0.001*\"da\" + 0.000*\"cashback\" + 0.000*\"highly_recomended\""
 ],
 [
  1,
  "0.149*\"excellent\" + 0.143*\"love\" + 0.134*\"convenient\" + 0.129*\"awesome\" + 0.093*\"helpful\" + 0.053*\"usefull\" + 0.044*\"fast\" + 0.036*\"service\" + 0.034*\"thanks\" + 0.025*\"far\""
 ],
 [
  2,
  "0.206*\"useful\" + 0.192*\"ok\" + 0.039*\"wow\" + 0.024*\"experience\" + 0.017*\"awsome\" + 0.015*\"excelent\" + 0.015*\"perfect\" + 0.014*\"maganda\" + 0.013*\"well\" + 0.011*\"superb\""
 ],
 [
  3,
  "0.722*\"good\" + 0.095*\"amazing\" + 0.032*\"happy\" + 0.024*\"convinient\" + 0.018*\"okay\" + 0.011*\"nc\" + 0.001*\"contented\" + 0.000*\"kaau\" + 0.000*\"highly_recomended\" + 0.000*\"verrygood\""
 ],
 [
  4,
  "0.031*\"verry\" + 0.031*\"bills\" + 0.031*\"load\" + 0.029*\"pay\" + 0.025*\"super\" + 0.022*\"transaction\" + 0.021*\"application\" + 0.020*\"free\" + 0.019

##### Optimizing Number of Topics for Bad Reviews

In [70]:
# Build LDA model
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=b_corpus,
                                           id2word=b_id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [71]:
print(json.dumps(lda_model2.print_topics(), indent=1))

[
 [
  0,
  "0.088*\"concern\" + 0.046*\"last_year\" + 0.036*\"locked\" + 0.035*\"usefull\" + 0.034*\"empty\" + 0.033*\"acount\" + 0.033*\"daming\" + 0.022*\"basically\" + 0.021*\"msg\" + 0.021*\"everytime\""
 ],
 [
  1,
  "0.249*\"pretty\" + 0.035*\"almost\" + 0.035*\"super\" + 0.033*\"apaka\" + 0.033*\"pasok\" + 0.030*\"already\" + 0.029*\"ayaw_gumana\" + 0.027*\"ins\" + 0.027*\"lagi\" + 0.025*\"grabe\""
 ],
 [
  2,
  "0.127*\"payment\" + 0.064*\"kaltas\" + 0.027*\"yet\" + 0.025*\"emergency\" + 0.025*\"needed\" + 0.024*\"plz\" + 0.023*\"international\" + 0.023*\"beses\" + 0.021*\"welcome_gift\" + 0.020*\"latest_version\""
 ],
 [
  3,
  "0.084*\"linked\" + 0.056*\"tells\" + 0.049*\"rating\" + 0.030*\"id\" + 0.029*\"week\" + 0.029*\"tagal\" + 0.027*\"mean\" + 0.019*\"new\" + 0.019*\"babayaran\" + 0.019*\"however\""
 ],
 [
  4,
  "0.146*\"frustrated\" + 0.074*\"broken\" + 0.069*\"change\" + 0.056*\"able\" + 0.030*\"given\" + 0.026*\"wish\" + 0.024*\"text_message\" + 0.021*\"stupid\" + 0

In [72]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=gr_words_trigrams, dictionary=g_id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

#### Visualizing LDA model results

In [73]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, g_corpus, g_id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.370427 -0.232156       1        1  25.403717
0      0.154104  0.408080       2        1  22.003904
5     -0.129623 -0.038918       3        1  17.675848
1     -0.128776 -0.047469       4        1  12.574022
4     -0.129373 -0.039676       5        1  12.562101
3     -0.136759 -0.049862       6        1   9.780408, topic_info=           Term          Freq         Total Category  logprob  loglift
0          good  11137.000000  11137.000000  Default  30.0000  30.0000
12         nice   7600.000000   7600.000000  Default  29.0000  29.0000
67        great   4254.000000   4254.000000  Default  28.0000  28.0000
1     excellent   2409.000000   2409.000000  Default  27.0000  27.0000
38   convenient   2034.000000   2034.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
565         mas     57.229110     58.037187   Topic6  -5.0649   2.3108
428        user     56.619498     57.427569   Topic6  -5.0756   2.3106
429        real     54.432605     55.242546   Topic6  -5.1150   2.3100
891     alright     54.646679     55.468342   Topic6  -5.1110   2.3099
899    freebies     78.668657    108.615572   Topic6  -4.7467   2.0022

[214 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
583       1  0.978948  absolutely
286       4  0.993044      access
510       6  0.997149  accessible
13        5  0.996722     account
1004      2  0.960238  affordable
...     ...       ...         ...
228       2  0.944881       works
202       4  0.997498         wow
443       6  0.989222        yeah
137       4  0.981085         yes
130       2  0.988175         yet

[191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 2, 5, 4])

In [74]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, b_corpus, b_id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.313169  0.276058       1        1  24.813203
1      0.343313  0.241755       2        1  18.663212
4      0.018014 -0.086208       3        1  16.198652
0     -0.003207 -0.225749       4        1  15.670886
2     -0.011521 -0.118566       5        1  13.531850
5     -0.033429 -0.087290       6        1  11.122197, topic_info=           Term         Freq        Total Category  logprob  loglift
16       pretty  2799.000000  2799.000000  Default  30.0000  30.0000
62   frustrated  1429.000000  1429.000000  Default  29.0000  29.0000
87      payment  1035.000000  1035.000000  Default  28.0000  28.0000
13       linked  1250.000000  1250.000000  Default  27.0000  27.0000
4       concern   829.000000   829.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
80        since    65.336200    66.112940   Topic6  -4.6307   2.1844
193         idk    64.384959    65.165108   Topic6  -4.6454   2.1842
66     accounts    63.739274    64.512267   Topic6  -4.6554   2.1842
622       ngaun    61.194122    61.971563   Topic6  -4.6962   2.1836
544   live_chat    56.729491    57.506763   Topic6  -4.7719   2.1826

[213 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         3  0.998002          able
101       3  0.997306       account
66        6  0.992059      accounts
550       4  0.996338        acount
1235      6  0.989397      advisory
...     ...       ...           ...
1078      5  0.994330  welcome_gift
24        4  0.997596          well
376       3  0.995151          wish
1037      2  0.996243     worthless
99        5  0.998790           yet

[185 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 1, 3, 6])